In [3]:
import os, shutil

import numpy as np
from imageio import imread
import matplotlib.pyplot as plt

%matplotlib qt

In [4]:
from optv.imgcoord import image_coordinates
from optv.transforms import convert_arr_metric_to_pixel
from optv.orientation import match_detection_to_ref
from optv.segmentation import target_recognition
from optv.orientation import external_calibration, full_calibration
from optv.calibration import Calibration
from optv.tracking_framebuf import TargetArray
from optv.parameters import ControlParams, VolumeParams, TrackingParams, \
    SequenceParams, TargetParams

In [5]:
from pyptv import ptv
from pyptv import parameters as par
from pyptv import parameter_gui as pargui

In [6]:
# set paths:

active_path = '/Users/alex/Documents/OpenPTV/test_cavity/parametersRun1' # like from GUI
print(active_path)
working_folder = os.path.split(active_path)[0]
print(working_folder)
par_path = os.path.join(working_folder, 'parameters')
print(os.path.abspath(par_path))

# set folders, read number of cameras
par.copy_params_dir(active_path, par_path)


/Users/alex/Documents/OpenPTV/test_cavity/parametersRun1
/Users/alex/Documents/OpenPTV/test_cavity
/Users/alex/Documents/OpenPTV/test_cavity/parameters
copy from /Users/alex/Documents/OpenPTV/test_cavity/parametersRun1 to /Users/alex/Documents/OpenPTV/test_cavity/parameters


In [7]:
# read parameters
# first read the number of cameras 
# then read all the parameters, some use liboptv, some use pyptv

os.chdir(os.path.abspath(working_folder))
print("Inside a working folder: ", os.getcwd())

with open(os.path.join(par_path, 'ptv.par'), 'r') as f:
    n_cams = int(f.readline())

print(n_cams)

Inside a working folder:  /Users/alex/Documents/OpenPTV/test_cavity
4


In [8]:
print("Copying man_ori.dat from %s to %s..." % (par_path, working_folder))
try:
    if os.path.isfile(os.path.join(par_path, 'man_ori.dat')):
        shutil.copyfile(os.path.join(par_path, 'man_ori.dat'),
                        os.path.join(working_folder, 'man_ori.dat'))
        print("Copied man_ori.dat \n")
except:
    raise IOError('could not copy man_ori.dat')

Copying man_ori.dat from /Users/alex/Documents/OpenPTV/test_cavity/parameters to /Users/alex/Documents/OpenPTV/test_cavity...
Copied man_ori.dat 



In [9]:
def read_parameters():
    """ reads all the parameters from the /parameters in the working folder"""


    # Control parameters
    cpar = ControlParams(n_cams)
    cpar.read_control_par(b'parameters/ptv.par')

    # Sequence parameters
    spar = SequenceParams(num_cams=n_cams)
    spar.read_sequence_par(b'parameters/sequence.par',n_cams)

    # Volume parameters
    vpar = VolumeParams()
    vpar.read_volume_par(b'parameters/criteria.par')

    # Tracking parameters
    track_par = TrackingParams()
    track_par.read_track_par(b'parameters/track.par')

    # Target parameters
    tpar = TargetParams(n_cams)
    # tpar.read(b'parameters/targ_rec.par') # for all images

    tpar.read(b'parameters/detect_plate.par') # for calibration images


    # Examine parameters, multiplane (single plane vs combined calibration)
    epar= par.ExamineParams()
    epar.read()

    pftVersionParams = par.PftVersionParams(par_path)
    pftVersionParams.read()
    Existing_Target = np.bool(pftVersionParams.Existing_Target)

    calOriParams = par.CalOriParams(n_cams, path=par_path)
    calOriParams.read()

    # Calibration parameters from ORI and ADDPAR files
    cals =[]
    for i_cam in range(n_cams):
        cal = Calibration()
        tmp = cpar.get_cal_img_base_name(i_cam)
        cal.from_file(tmp + b'.ori', tmp + b'.addpar')
        cals.append(cal)

    return cpar, spar, vpar, track_par, tpar, cals, epar, Existing_Target, calOriParams

In [10]:
cpar, spar, vpar, track_par, tpar, cals, epar, Existing_Target, calOriParams = read_parameters()

In [11]:
def rgb2gray(rgb):
    return (0.2989 * rgb[:,:,0] + 0.5870 * rgb[:,:,1] + 0.1140 * rgb[:,:,2]).clip(0,255).astype(np.uint8)

In [12]:
# read calibration images
cal_images = []
for i in range(n_cams):
    imname = cpar.get_cal_img_base_name(i)
    # imname = calParams.img_cal_name[i]
    # for imname in self.calParams.img_cal_name:
    # self.cal_images.append(imread(imname))
    im = imread(imname.decode())
    if im.ndim > 2:
        im = rgb2gray(im)

    cal_images.append(im)

In [ ]:
def show_cal_images(ind = None):
    """ Show calibration images, if no inputs, all images are shown by default 
    if input is a scalar - only that image is shown 
    if input is a set of indices, only those cal images are shown
    
    Example:
        show_cal_images()
        show_cal_images(2)
        show_cal_images([0,2])
    """
    
    fig_list = list()
    
    if ind is None: # all cameras
        ind = range(n_cams)
    else:
        ind = np.atleast_1d(ind)
        
    if len(ind) > 1:
        for counter, value in enumerate(ind):
            fig, ax = plt.subplots()
            ax.imshow(cal_images[value],cmap=plt.cm.gray)
            plt.title('Camera {0}'.format(value))
            plt.show()
            fig_list.append([fig,ax])
    else: # likely a scalar
        fig, ax = plt.subplots()
        ax.imshow(cal_images[ind[0]],cmap=plt.cm.gray)
        plt.title('Camera {0}'.format(ind[0]))
        plt.show()
        fig_list.append([fig,ax])
        
    
    return fig_list

In [48]:
# show_cal_images()
show_cal_images([0,3])
# show_cal_images([2])

[[<Figure size 1280x960 with 1 Axes>,
 [<Figure size 1280x960 with 1 Axes>,

In [49]:
from optv.image_processing import preprocess_image

if cpar.get_hp_flag(): # if highpass
    for i in range(n_cams):
        cal_images[i] = preprocess_image(cal_images[i], 0, cpar, 25)
    
    # reset the hp_flag
    cpar.set_hp_flag(False)

In [50]:
print(tpar.get_grey_thresholds())

[40 40 40 40]


In [51]:
from optv.correspondences import correspondences, MatchedCoords
from optv.segmentation import target_recognition


detections, corrected = [],[]
for i_cam, img in enumerate(cal_images):
    
    targs = target_recognition(img, tpar, i_cam, cpar)

    targs.sort_y()
    detections.append(targs)
    mc = MatchedCoords(targs, cpar, cals[i_cam])
    corrected.append(mc)

In [52]:
x = [[i.pos()[0] for i in row] for row in detections]
y = [[i.pos()[1] for i in row] for row in detections]

In [54]:
f = show_cal_images() # show images, overlay with dots
for i in range(n_cams):
    f[i][1].scatter(x[i],y[i],marker='+',color='b')

In [55]:
def read_cal_block():
    # type: () -> numpy.ndarray
    return np.atleast_1d(np.loadtxt(calOriParams.fixp_name, dtype=[('id', 'i4'), ('pos', '3f8')], skiprows=0))

In [56]:
cal_points = read_cal_block()
print(cal_points[0], '...', cal_points[-1])

(1, [-40., -25.,   8.]) ... (73, [30., 25.,  8.])


In [58]:
# project initial guess
f = show_cal_images()

for i_cam in range(n_cams):
    x, y = [], []
    for row in cal_points:
        projected = image_coordinates(np.atleast_2d(row['pos']), \
                                      cals[i_cam], cpar.get_multimedia_params())
        pos = convert_arr_metric_to_pixel(projected, cpar)

        x.append(pos[0][0])
        y.append(pos[0][1])
        
    f[i_cam][1].scatter(x,y,marker='+',color='yellow')

In [22]:
pos0 = cals[0].get_pos()
def initial_guess(i_cam=0, pos = pos0):
    fig, ax = plt.subplots(figsize=(10,10))
    # change the position
    # pos = cals[i_cam].get_pos()
    cals[i_cam].set_pos(pos)
    
    # project and draw
    x, y = [], []
    for row in cal_points:
        projected = image_coordinates(np.atleast_2d(row['pos']), \
                                      cals[i_cam], cpar.get_multimedia_params())
        pos = convert_arr_metric_to_pixel(projected, cpar)

        x.append(pos[0][0])
        y.append(pos[0][1])

    # for i in range(cpar.get_num_cams()):
    ax.imshow(cal_images[i_cam],cmap=plt.cm.gray)
    ax.scatter(x,y,marker='+',color='yellow')
    plt.show()

In [25]:
def edit_cal_parameters():
    cp = pargui.Calib_Params(par_path=par_path)
    cp.edit_traits(kind='modal')
    
    # at the end of a modification, copy the parameters
    par.copy_params_dir(par_path, active_path)

In [26]:
edit_cal_parameters()

# must read again the parameters
cpar, spar, vpar, track_par, tpar, cals, epar, Existing_Target,\
                            calOriParams = read_parameters()

calOriParams.fixp_name # test

inside CalHandler  /Users/alex/Documents/OpenPTV/test_cavity/parameters
copy from /Users/alex/Documents/OpenPTV/test_cavity/parameters to /Users/alex/Documents/OpenPTV/test_cavity/parametersRun1


'cal/target_on_a_side.txt'

In [86]:
# to click 4 points on every image to get the manual orientation
# the points are written in the man_ori array, see above
# the points in pixels are stored in man_ori.dat file that
# should appear in the working_folder and later copied to the 
# parameters folder active_path for the future 


man_ori_par = np.loadtxt(os.path.join(par_path, 'man_ori.par')).reshape((n_cams,-1)).astype(np.int)

class Click():
    def __init__(self, ax, func, man_points, button = 1):
        self.ax = ax
        self.func = func
        self.button = button
        self.man_points = man_points
        self.press = False
        self.move = False
        self.data = []
        self.c1 = self.ax.figure.canvas.mpl_connect('button_press_event', self.onpress)
        self.c2 = self.ax.figure.canvas.mpl_connect('button_release_event', self.onrelease)
        self.c3 = self.ax.figure.canvas.mpl_connect('motion_notify_event', self.onmove)

    def onclick(self,event):
        if event.inaxes == self.ax:
            if event.button == self.button:
                man_point = self.man_points.pop(0)
                self.func(event, self.ax, man_point)
                self.data.append([man_point,event.xdata,event.ydata])
    def onpress(self,event):
        self.press = True
    def onmove(self,event):
        if self.press:
            self.move = True
    def onrelease(self,event):
        if self.press and not self.move:
            self.onclick(event)
        
        self.press = False 
        self.move = False


def pick(event, ax, man_point):
    # print(event.xdata, event.ydata, man_point)
    ax.scatter(event.xdata, event.ydata)
    ax.annotate(str(man_point),(event.xdata,event.ydata),color='r')
    ax.figure.canvas.draw()

## coulnd't find the way to make it a loop

In [87]:
man_points = []
f = show_cal_images(0)
click = Click(f[0][1], pick, man_ori_par[0].tolist(), button = 1)
man_points.append(click.data)

In [88]:
f = show_cal_images(1)
click = Click(f[0][1], pick, man_ori_par[1].tolist(), button = 1)
man_points.append(click.data)

In [89]:
f = show_cal_images(2)
click = Click(f[0][1], pick, man_ori_par[2].tolist(), button = 1)
man_points.append(click.data)

In [90]:
f = show_cal_images(3)
click = Click(f[0][1], pick, man_ori_par[3].tolist(), button = 1)
man_points.append(click.data)

In [91]:
man_points = np.array(man_points).reshape((-1,3)
print(man_points)

[[[3, 226.68614718614714, 335.8463203463202], [5, 977.5086580086584, 333.0757575757574], [72, 1016.2965367965371, 604.590909090909], [73, 257.1623376623377, 621.2142857142856]], [[3, 287.63852813852816, 352.46969696969677], [5, 1016.2965367965371, 333.0757575757574], [72, 996.9025974025976, 612.9025974025973], [73, 262.7034632034631, 621.2142857142856]], [[3, 234.99783549783547, 391.25757575757564], [5, 969.19696969697, 380.17532467532453], [72, 969.19696969697, 895.4999999999999], [73, 243.30952380952374, 920.4350649350648]], [[3, 251.62121212121212, 297.05844155844136], [5, 988.5909090909095, 294.28787878787864], [72, 985.8203463203465, 834.5476190476189], [73, 257.1623376623377, 815.1536796536795]]]


In [122]:
np.savetxt('new_man_ori.dat',man_points,fmt='%d %.2f %.2f')

In [124]:
tmp = np.loadtxt('new_man_ori.dat')
tmp

array([[   3.  ,  226.69,  335.85],
       [   5.  ,  977.51,  333.08],
       [  72.  , 1016.3 ,  604.59],
       [  73.  ,  257.16,  621.21],
       [   3.  ,  287.64,  352.47],
       [   5.  , 1016.3 ,  333.08],
       [  72.  ,  996.9 ,  612.9 ],
       [  73.  ,  262.7 ,  621.21],
       [   3.  ,  235.  ,  391.26],
       [   5.  ,  969.2 ,  380.18],
       [  72.  ,  969.2 ,  895.5 ],
       [  73.  ,  243.31,  920.44],
       [   3.  ,  251.62,  297.06],
       [   5.  ,  988.59,  294.29],
       [  72.  ,  985.82,  834.55],
       [  73.  ,  257.16,  815.15]])

In [126]:
# orientation with file 
# load the dots that are stored in the man_ori.dat (pixels)
# and their indices from the man_ori.par (index according to the calibration block text file)
# and plots over the image
man_ori_par = np.loadtxt(os.path.join(par_path, 'man_ori.par')).reshape((n_cams,-1)).astype(np.int)
man_ori_dat = np.loadtxt(os.path.join(working_folder, 'man_ori.dat')).reshape((n_cams,-1))

# or simply read the new_man_ori.dat that has both man_ori.par and man_ori.dat
# 
# tmp = np.loadtxt('new_man_ori.dat')
# print(tmp)

f = show_cal_images()
for i in range(n_cams):
    for j in range(4):
        f[i][1].scatter(man_ori_dat[i][j*2],man_ori_dat[i][j*2+1],color='y',marker='x')
        f[i][1].annotate(str(man_ori_par[i][j]),(man_ori_dat[i][j*2],man_ori_dat[i][j*2+1]),color='r')

# backup to the parameters folder 
shutil.copyfile(os.path.join(working_folder, 'man_ori.dat'), os.path.join(active_path, 'man_ori.dat'))

'/Users/alex/Documents/OpenPTV/test_cavity/parametersRun1/man_ori.dat'

In [128]:
f = show_cal_images()

sorted_targs = []

for i_cam in range(n_cams):

    # if len(self.cal_points) > len(self.detections[i_cam]):
    #     raise ValueError("Insufficient detected points, need at least as"
    #                  "many as fixed points")

    targs = match_detection_to_ref(cals[i_cam], cal_points['pos'],
                                   detections[i_cam], cpar)
    x, y, pnr = [], [], []
    for t in targs:
        if t.pnr() != -999:
            # pnr.append(cal_points['id'][t.pnr()])
            # x.append(t.pos()[0])
            # y.append(t.pos()[1])
            f[i_cam][1].scatter(t.pos()[0], t.pos()[1],1,color='yellow')
            f[i_cam][1].annotate(str(cal_points['id'][t.pnr()]),(t.pos()[0],t.pos()[1]),color='r')

    sorted_targs.append(targs)
    # ax[i_cam].plot_num_overlay(x, y, pnr) # <-- implement this one text over image
    # ax[i_cam].scatter(x,y,1,color='yellow')
    

In [131]:
def backup_ori_files():
    # backup ORI files
    for f in calOriParams.img_ori[:n_cams]:
        shutil.copyfile(f, f + '.bck')
        g = f.replace('ori', 'addpar')
        shutil.copyfile(g, g + '.bck')

In [157]:
def _write_ori(i_cam):
    """ Writes ORI and ADDPAR files for a single calibration result
    """

    ori = calOriParams.img_ori[i_cam]
    addpar = ori.replace('ori', 'addpar')
    print("Saving:", ori, addpar)
    cals[i_cam].write(ori.encode(), addpar.encode())
    if epar.Examine_Flag and not epar.Combine_Flag:
        save_point_sets(i_cam)

In [158]:
def save_point_sets(i_cam):
    """
    Saves detected and known calibration points in crd and fix format, respectively.
    These files are needed for multiplane calibration.
    """

    ori = calOriParams.img_ori[i_cam]
    txt_detected = ori.replace('ori', 'crd')
    txt_matched = ori.replace('ori', 'fix')


    detected, known = [],[]
    targs = sorted_targs[i_cam]
    for i,t in enumerate(targs):
        if t.pnr() != -999:
            detected.append(t.pos())
            known.append(cal_points['pos'][i]) 
    nums = np.arange(len(detected))
    # for pnr in nums:
    #     print(targs[pnr].pnr())
    #     print(targs[pnr].pos())
    #   detected[pnr] = targs[pnr].pos()

    detected = np.hstack((nums[:,None], np.array(detected)))
    known = np.hstack((nums[:,None], np.array(known)))

    np.savetxt(txt_detected, detected, fmt="%9.5f")
    np.savetxt(txt_matched, known, fmt="%10.5f")

In [159]:
def reproject_cal_points(i_cam=0):
    
    # read cals again, after raw_orient
    cal = Calibration()
    tmp = cpar.get_cal_img_base_name(i_cam)
    cal.from_file(tmp + b'.ori', tmp + b'.addpar')
    cals[i_cam] = cal
        
    # project initial guess
    f = show_cal_images(i_cam)

    x, y = [], []
    for row in cal_points:
        projected = image_coordinates(np.atleast_2d(row['pos']), \
                                      cals[i_cam], cpar.get_multimedia_params())
        pos = convert_arr_metric_to_pixel(projected, cpar)

        x.append(pos[0][0])
        y.append(pos[0][1])

    f[0][1].scatter(x,y,marker='+',color='yellow')

In [167]:
# def raw_orient():
"""
update the external calibration with results of raw orientation, i.e.
the iterative process that adjust the initial guess' external
parameters (position and angle of cameras) without internal or
distortions.

See: https://github.com/openptv/openptv/liboptv/src/orientation.c#L591
"""

# backup the ORI/ADDPAR files first
backup_ori_files()

man_ori_dat = np.loadtxt(os.path.join(working_folder, 'man_ori.dat')).reshape((n_cams,-1))

# get manual points from cal_points and use ids from man_ori.par

for i_cam in range(n_cams):
    selected_points = np.zeros((4, 3))
    for i, cp_id in enumerate(cal_points['id']):
        for j in range(4):
            if cp_id == man_ori_par[i_cam][j]:
                selected_points[j, :] = cal_points['pos'][i, :]
                continue

    # in pixels:
    # manual_detection_points = np.array((camera[i_cam]._x, camera[i_cam]._y)).T
    manual_detection_points = man_ori_dat[i_cam].reshape((-1,2)).T
    
    print(selected_points)
    print(manual_detection_points)

    success = external_calibration(cals[i_cam], selected_points, \
                                   manual_detection_points, cpar)

    print(success)

    if success is False:
        print("Initial guess was not successful \n")
    else:
        reproject_cal_points(i_cam)
        _write_ori(i_cam)

[[-40.   0.  -8.]
 [-40.  25.   8.]
 [ 30.   0.  -8.]
 [ 30.  25.   8.]]
[[1009.  979.  246.  235.]
 [ 608.  335.  620.  344.]]
False
Initial guess was not successful 

[[-40.   0.  -8.]
 [-40.  25.   8.]
 [ 30.   0.  -8.]
 [ 30.  25.   8.]]
[[1002. 1013.  261.  285.]
 [ 609.  335.  620.  355.]]
True
Saving: cal/cam2.tif.ori cal/cam2.tif.addpar
[[-40.   0.  -8.]
 [-40.  25.   8.]
 [ 30.   0.  -8.]
 [ 30.  25.   8.]]
[[245. 236. 967. 970.]
 [926. 395. 892. 382.]]
False
Initial guess was not successful 

[[-40.   0.  -8.]
 [-40.  25.   8.]
 [ 30.   0.  -8.]
 [ 30.  25.   8.]]
[[262. 251. 989. 988.]
 [823. 300. 837. 299.]]
False
Initial guess was not successful 



In [198]:
# fine orientation

scale = 5000


# backup the ORI/ADDPAR files first
backup_ori_files()

op = par.OrientParams()
op.read()

# recognized names for the flags:
names = ['cc', 'xh', 'yh', 'k1', 'k2', 'k3', 'p1', 'p2', 'scale', 'shear']
op_names = [op.cc, op.xh, op.yh, op.k1, op.k2, op.k3, op.p1, op.p2, op.scale, op.shear]

flags = []
for name, op_name in zip(names, op_names):
    if (op_name == 1):
        flags.append(name)


for i_cam in range(n_cams): # iterate over all cameras

    if epar.Combine_Flag:
        """ Performs multiplane calibration, in which for all cameras the 
        pre-processed planes in multi_plane.par combined.
        Overwrites the ori and addpar files of the cameras specified 
        in cal_ori.par of the multiplane parameter folder
        """

        all_known = []
        all_detected = []

        for i in range(MultiParams.n_planes): # combine all single planes

            # c = self.calParams.img_ori[i_cam][-9] # Get camera id
            c = re.findall('\\d+',calOriParams.img_ori[i_cam])[0] # not all ends with a number

            file_known = MultiParams.plane_name[i]+c+'.tif.fix'
            file_detected = MultiParams.plane_name[i]+c+'.tif.crd'

            # Load calibration point information from plane i
            try:
                known = np.loadtxt(file_known)
                detected = np.loadtxt(file_detected)
            except:
                raise IOError("reading {} or {} failed".format(file_known,file_detected))


            if np.any(detected == -999):
                raise ValueError(("Using undetected points in {} will cause " +
                "silliness. Quitting.").format(file_detected))

            num_known = len(known)
            num_detect = len(detected)

            if num_known != num_detect:
                raise ValueError("Number of detected points (%d) does not match" +\
                " number of known points (%d) for %s, %s" % \
                (num_known, num_detect, file_known, file_detected))

            if len(all_known) > 0:
                detected[:,0] = all_detected[-1][-1,0] + 1 + np.arange(len(detected))

            # Append to list of total known and detected points
            all_known.append(known)
            all_detected.append(detected)


        # Make into the format needed for full_calibration.
        all_known = np.vstack(all_known)[:,1:]
        all_detected = np.vstack(all_detected)

        # this is the main difference in the multiplane mode
        # that we fill the targs and cal_points by the 
        # combined information

        targs = TargetArray(len(all_detected))
        for tix in range(len(all_detected)):
            targ = targs[tix]
            det = all_detected[tix]

            targ.set_pnr(tix)
            targ.set_pos(det[1:])

        cal_points = np.empty((all_known.shape[0],)).astype(dtype=[('id', 'i4'), ('pos', '3f8')])
        cal_points['pos'] = all_known
    else:
        targs = sorted_targs[i_cam]

    try: 
        residuals, targ_ix, err_est = full_calibration(cals[i_cam], cal_points['pos'], \
                                                   targs, cpar, flags)
    except:
        raise ValueError("full calibration failed\n")
    # save the results
    _write_ori(i_cam)


    # Plot the output
    # self.reset_plots()

    x, y = [], []
    for r, t in zip(residuals, targ_ix):
        if t != -999:
            pos = targs[t].pos()
            x.append(pos[0])
            y.append(pos[1])

    x, y = np.array(x), np.array(y)
    f = show_cal_images(i_cam)
    f[0][1].scatter(x, y, 1, color='orange')
    
    f[0][1].quiver(x, y, scale*residuals[:len(x),0], scale*residuals[:len(x),1],color='red',width=0.0025,headaxislength=0)

Saving: cal/cam1.tif.ori cal/cam1.tif.addpar
Saving: cal/cam2.tif.ori cal/cam2.tif.addpar
Saving: cal/cam3.tif.ori cal/cam3.tif.addpar


ValueError: full calibration failed


In [43]:
def _button_edit_ori_files_fired(self):
    editor = codeEditor(path=self.par_path)
    editor.edit_traits(kind='livemodal')

In [ ]:
# restore ori files
def restore_ori_files(n_cams, calOriParams):
    for f in calOriParams.img_ori[:n_cams]:
        print('restored %s ' % f)
        shutil.copyfile(f + '.bck', f)
        g = f.replace('ori', 'addpar')
        shutil.copyfile(g + '.bck', g)

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

In [ ]:
from ipywidgets import FloatSlider
x_widget = FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget);

In [21]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets